### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
week_str = today.strftime('%Y-%U')
today, week_str

(datetime.date(2022, 6, 11), '2022-23')

### Process after last business day, today must be last business day

In [2]:
today = today - timedelta(days=1)
wkago = today - timedelta(days=8)
today, wkago

(datetime.date(2022, 6, 10), datetime.date(2022, 6, 2))

### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [3]:
cols = 'name price_d price_w percent perform mrkt '.split()

In [4]:
format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,TOP,2022-06-10,57.50


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'TOP'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,555,TOP,SET50 / SETCLMV / SETTHSI,56.75,62.25,42.50,7.06,0.91,999.99,"115,771.58","3,166,012.86",3.52,555,2017-07-23 07:25:41.632818,2022-06-11 09:46:31.151042


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "TOP" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,TOP,2022-06-10,56.75,59.00,56.50,"55,079,884",59.00
1,TOP,2022-06-09,60.75,62.25,60.50,"29,941,500",61.00
2,TOP,2022-06-08,60.75,61.00,59.50,"19,039,094",60.00
3,TOP,2022-06-07,59.50,59.75,57.75,"17,958,932",58.00
4,TOP,2022-06-06,58.25,59.00,57.50,"19,372,547",58.75


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (today, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2022-06-10' AND name IN ('ASP', 'BCH', 'BDMS', 'DCC', 'DIF', 'DOHOME', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SCCC', 'SENA', 'SIS', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TOP', 'TPIPL', 'TTB', 'WHAIR', 'WHART')
ORDER BY name, date


27

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (wkago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2022-06-02' AND name IN ('ASP', 'BCH', 'BDMS', 'DCC', 'DIF', 'DOHOME', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SCCC', 'SENA', 'SIS', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TOP', 'TPIPL', 'TTB', 'WHAIR', 'WHART') 
ORDER BY name


27

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(27, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [13]:
trend

,name,price_d,price_w,percent
0,ASP,3.22,3.28,-1.83
1,BCH,19.10,19.90,-4.02
2,BDMS,24.80,25.75,-3.69
3,DCC,2.82,2.78,1.44
4,DIF,13.50,13.70,-1.46
5,DOHOME,18.50,18.90,-2.12
6,JASIF,10.90,10.90,0.00
7,KCE,64.25,65.75,-2.28
8,MAKRO,35.25,37.00,-4.73
9,MCS,12.30,12.50,-1.60


In [14]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [15]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
18,STA,22.60,25.00,-9.60%,Worse
13,RCL,43.00,46.00,-6.52%,Worse
19,SYNEX,20.20,21.30,-5.16%,Worse
8,MAKRO,35.25,37.00,-4.73%,Worse
1,BCH,19.10,19.90,-4.02%,Worse
2,BDMS,24.80,25.75,-3.69%,Worse
10,NER,6.60,6.80,-2.94%,Worse
11,ORI,10.30,10.60,-2.83%,Worse
22,TOP,56.75,58.25,-2.58%,Worse
25,WHAIR,8.20,8.40,-2.38%,Worse


In [16]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,88.89%
Better,7.41%
No Change,3.70%


In [17]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(249, 8)

In [18]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [19]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(27, 13)

In [20]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
2,BDMS,24.80,25.75,-3.69%,Worse,SET50
22,TOP,56.75,58.25,-2.58%,Worse,SET50
7,KCE,64.25,65.75,-2.28%,Worse,SET50
12,PTTGC,47.25,48.25,-2.07%,Worse,SET50
14,SCC,363.00,369.00,-1.63%,Worse,SET50
20,TISCO,89.50,90.75,-1.38%,Worse,SET50
24,TTB,1.27,1.24,2.42%,Better,SET50


In [21]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,85.71%
Better,14.29%


In [22]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
18,STA,22.60,25.00,-9.60%,Worse,SET100
13,RCL,43.00,46.00,-6.52%,Worse,SET100
19,SYNEX,20.20,21.30,-5.16%,Worse,SET100
1,BCH,19.10,19.90,-4.02%,Worse,SET100
11,ORI,10.30,10.60,-2.83%,Worse,SET100
5,DOHOME,18.50,18.90,-2.12%,Worse,SET100


In [23]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,100.00%


In [24]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
8,MAKRO,35.25,37.00,-4.73%,Worse,SET999
10,NER,6.60,6.80,-2.94%,Worse,SET999
25,WHAIR,8.20,8.40,-2.38%,Worse,SET999
0,ASP,3.22,3.28,-1.83%,Worse,SET999
17,SIS,28.50,29.00,-1.72%,Worse,SET999
9,MCS,12.30,12.50,-1.60%,Worse,SET999
4,DIF,13.50,13.70,-1.46%,Worse,SET999
21,TMT,9.30,9.40,-1.06%,Worse,SET999
26,WHART,10.10,10.20,-0.98%,Worse,SET999
16,SENA,4.10,4.14,-0.97%,Worse,SET999


In [25]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Worse,85.71%
Better,7.14%
No Change,7.14%


In [26]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt


In [27]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
